In [1]:
import pandas as pd
import numpy as np

In [2]:
osoby = pd.read_csv('vstupy/osoby.unl',
                    encoding='cp1250', sep='|', header=None)
osoby.columns = 'id_osoba, pred, prijmeni, jmeno, za, \
narozeni, pohlavi, zmena, umrti, x'.split(', ')

In [3]:
osoby.head(3)

,id_osoba,pred,prijmeni,jmeno,za,narozeni,pohlavi,zmena,umrti,x
0,1,Ing.,Bachna,Petr,,08.01.1948,M,NaN,NaN,NaN
1,2,Ing.,Bečvář,Ivan,,19.01.1934,M,NaN,NaN,NaN
2,3,,Bejček,Josef,,03.03.1935,M,NaN,NaN,NaN


In [4]:
osoby.narozeni = ['{2}-{1}-{0}'.format(*j.split('.')) for j in osoby.narozeni]

In [5]:
zarazeni = pd.read_csv('vstupy/zarazeni.unl',
                    encoding='cp1250', sep='|', header=None)
zarazeni.columns = 'id_osoba, id_of, cl_funkce, od_o, do_o, od_f, do_f, x'.split(', ')

In [6]:
zarazeni = zarazeni.loc[zarazeni.cl_funkce == 0]

In [7]:
organy = pd.read_csv('vstupy/organy.unl',
                    encoding='cp1250', sep='|', header=None)
organy.columns = 'id_organ, organ_id_organ, id_typ_organu, zkratka, nazev_organu_cz, nazev_organu_en, od_organ, \
do_organ, priorita, cl_organ_base, x'.split(', ')

### mapovani

In [8]:
zarm = zarazeni.merge(organy, left_on='id_of', right_on='id_organ')

In [9]:
zpsp = zarm.loc[zarm.nazev_organu_cz.\
                isin(['Poslanecká sněmovna',\
                     'Senát', 'Vláda České republiky'])]
zpsp.head(2)

,id_osoba,id_of,cl_funkce,od_o,do_o,od_f,do_f,x_x,id_organ,organ_id_organ,id_typ_organu,zkratka,nazev_organu_cz,nazev_organu_en,od_organ,do_organ,priorita,cl_organ_base,x_y
127,1,165,0,1992-06-06 00,1996-06-06 00,NaN,NaN,NaN,165,0.0,11,PSP1,Poslanecká sněmovna,Chamber of Deputies,06.06.1992,06.06.1996,1000.0,0.0,NaN
128,2,165,0,1992-06-06 00,1996-06-06 00,NaN,NaN,NaN,165,0.0,11,PSP1,Poslanecká sněmovna,Chamber of Deputies,06.06.1992,06.06.1996,1000.0,0.0,NaN


In [10]:
for g in zpsp.groupby('zkratka'):
    osoby.loc[osoby.id_osoba.isin(set(g[1].id_osoba.tolist()))].\
drop(['id_osoba', 'zmena', 'x'], axis=1).to_csv('vystupy/%s.csv' % g[0], index=False)

In [11]:
with open('org.txt', 'w') as fw:
    with open('vstupy/organy.unl', encoding='cp1250') as f:
        fw.write(f.read())